In [3]:
using JuMP, Gurobi
import GLPK
import Random
import Plots

In [4]:
function generate_distance_matrix(n; random_seed = 1)
    rng = Random.MersenneTwister(random_seed)
    X = 100 * rand(rng, n)
    Y = 100 * rand(rng, n)
    d = [round(Int, sqrt((X[i] - X[j])^2 + (Y[i] - Y[j])^2)) for i in 1:n, j in 1:n]
    return X, Y, d
end

n = 7
X, Y, d = generate_distance_matrix(n)
d

7×7 Matrix{Int64}:
  0  76   8  50  62  58  75
 76   0  74  46  23  22  80
  8  74   0  53  58  57  67
 50  46  53   0  50  24  97
 62  23  58  50   0  28  57
 58  22  57  24  28   0  82
 75  80  67  97  57  82   0

In [5]:
i = 5 # Replace this with your desired value of i
result = (1 << (i-1)) -1

println("Result in binary: ", string(result, base=2))

Result in binary: 1111


In [6]:
function tsp_dynamic_programming_memoization(distance_matrix)
    n = size(distance_matrix, 1)
    num_states = 2^n
    memo = Dict{Tuple{Int, Int}, Int}() # mask, pos를 key로 사용 

    return tsp(memo, distance_matrix, 1, 1, n)
end

function tsp(memo, distance_matrix, mask, pos, n)
    if haskey(memo, (mask, pos)) # 이미 계산된 부분인지 확인 
        return memo[(mask, pos)]
    end

    if mask == 2^n - 1 # 전부 다 방문함 
        return distance_matrix[pos, 1]
    end

    min_cost = Inf
    for next_city in 2:n
        if (mask & (1 << (next_city - 1))) == 0 # next_city is not visited.
            new_mask = mask | (1 << (next_city - 1))
            new_cost = distance_matrix[pos, next_city] + tsp(memo, distance_matrix, new_mask, next_city, n)
            min_cost = min(min_cost, new_cost)
        end
    end

    memo[(mask, pos)] = min_cost
    return min_cost
end

distances = d

result = tsp_dynamic_programming_memoization(distances)
println("Minimum tour length using memoization: $result")

Minimum tour length using memoization: 251.0


In [7]:
function tsp_dynamic_programming()
    memo = fill(Inf, n, 1 << n)
    memo[1, 1] = 0
    
    for mask in 1:(1 << n) - 1
        for s in 1:n
            if (mask & (1 << (s - 1))) != 0 # if city s is visited.
                for j in 2:n
                    if (mask & (1 << (j - 1))) == 0 # if city j is not visited.
                        #=
                        if memo[j, mask | (1 << (j - 1))] > memo[s, mask] + d[s, j] && memo[j, mask | (1 << (j - 1))] != Inf
                            println("$(memo[j, mask | (1 << (j - 1))]) vs $(memo[s, mask] + d[s, j])")
                        end
                        =#
                        memo[j, mask | (1 << (j - 1))] = min(memo[j, mask | (1 << (j - 1))], memo[s, mask] + d[s, j]) 
                    end
                end
            end
        end
    end
    
    ans = Inf
    for i in 2:n
        ans = min(ans, memo[i, (1 << n) - 1] + d[i, 1])
    end
    
    return ans
end

result = tsp_dynamic_programming()
println("The cost of the most efficient tour = $result")


The cost of the most efficient tour = 251.0
